In [1]:
from astropy import units as u

In [16]:
def convert_and_strip_units(quantity, output_unit=None, digit=10):
    """
    Strips units and return the numerical value.

    Parameters
    ----------
    quantity : int or float or list or None or ~astropy.units.quantity.Quantity
        Numerical quantity. Pass it without including units if default units are intended.
    output_unit : ~astropy.units.core.UnitBase or ~astropy.units.quantity.Quantity
        The default units in which the quantity is to be converted before extracting the value.

    Other Parameters
    ----------------
    
    round: int
        round to that number of digit after conversion. Default ``10``. Set to 
        ``None`` to disable.

    Returns
    -------
    int or float or None or ~numpy.ndarray
        The numerical value extracted from ``quantity``

    Examples
    --------
    
        >>> convert_and_strip_units(4.5 * u.um, u.nm)
        <<< 4500.0

        >>> convert_and_strip_units(200000 * (u.m)**-1, 1/u.cm)
        <<< 2000.0

    Raises
    ------
    TypeError
        Raised when ``quantity`` is a astropy.units quantity and ``output_unit`` is ``None``.
    
    """
    if isinstance(quantity, u.Quantity):
        if output_unit in (u.deg_C, u.imperial.deg_F, u.K):
            quantity = quantity.to_value(output_unit, equivalencies=u.temperature())
        elif isinstance(output_unit, (u.UnitBase, u.Quantity)):
            quantity = quantity.to_value(output_unit, equivalencies=u.spectral())
        else:
            raise TypeError(
                "'output_unit' parameter is not a valid astropy unit: {0}".format(
                    output_unit
                )
            )

        if digit:
            quantity = round(quantity, digit)

    return quantity

In [3]:
casu(10*u.cm, (u.m)**-1)

10.0

In [4]:
l = None

In [5]:
print(casu(l, u.cm))

None


In [6]:
print(type(casu(10, u.cm)))

<class 'int'>


In [7]:
lm = 18 * u.m

In [8]:
print(type(lm))

<class 'astropy.units.quantity.Quantity'>


In [9]:
type(10 * u.Unit("m-1"))

astropy.units.quantity.Quantity

In [11]:
def air2vacuum(wv):
    return 1.2*wv

In [12]:
def nm2cm(wv):
    return wv/(10**7)

In [26]:
l = 10* u.cm

In [27]:
print(type(l))

<class 'astropy.units.quantity.Quantity'>


In [29]:
l.unit.is_equivalent(u.m)

True

In [118]:
def get_waverange(
    wmin=None,
    wmax=None,
    wunit=None,
    wavenum_min=None,
    wavenum_max=None,
    wavelength_min=None,
    wavelength_max=None,
    medium="air",
):
    """ Returns wavenumber based on whatever input was given: either ν_min, ν_max 
    directly, or λ_min, λ_max  in the given propagation ``medium``.

    Parameters
    ----------

    medium: ``'air'``, ``'vacuum'``
        propagation medium

    wavenum_min, wavenum_max: float, or ~astropy.units.quantity.Quantity or ``None``
        wavenumbers

    wavelength_min, wavelength_max: float, or ~astropy.units.quantity.Quantity or ``None``
        wavelengths in given ``medium``

    Returns
    -------

    wavenum_min, wavenum_max,: float
        wavenumbers

    """

    # Check input
    if (isinstance(wmin, u.Quantity) or isinstance(wmin, u.Quantity)) and wunit is not None:
        raise ValueError("Cannot pass units with both wavelength and wave unit")

    represent_wavelength = False
    represent_wavenum = False
    
    if wunit is not None:
        assert wmin is not None and wmax is not None
        assert u.Unit(wunit).is_equivalent(u.m) or u.Unit(wunit).is_equivalent(1 / u.m)
        assert not isinstance(wmin, u.Quantity)
        assert not isinstance(wmax, u.Quantity)
        
        wmin = wmin * u.Unit(wunit)
        wmax = wmax * u.Unit(wunit)

    if wmin is not None or wmax is not None:
        assert wmin is not None and wmax is not None
        assert isinstance(wmin, u.Quantity) and isinstance(wmax, u.Quantity)
        assert wmin.unit.is_equivalent(u.m) or wmin.unit.is_equivalent(1 / u.m)
        assert wmax.unit.is_equivalent(u.m) or wmax.unit.is_equivalent(1 / u.m)
        assert wmin.unit.is_equivalent(wmax.unit)
        
        if wmin.unit.is_equivalent(u.m):
            represent_wavelength = True
            wmin = convert_and_strip_units(wmin, u.nm)
            wmax = convert_and_strip_units(wmax, u.nm)
        else:
            represent_wavenum = True
            wmin = convert_and_strip_units(wmin, 1 / u.cm)
            wmax = convert_and_strip_units(wmax, 1 / u.cm)
  
    wavelength_min = convert_and_strip_units(wavelength_min, u.nm)
    wavelength_max = convert_and_strip_units(wavelength_max, u.nm)
    wavenum_min = convert_and_strip_units(wavenum_min, 1 / u.cm)
    wavenum_max = convert_and_strip_units(wavenum_max, 1 / u.cm)
    if (
        wmin is None
        and wmax is None
        and wavelength_min is None
        and wavelength_max is None
        and wavenum_min is None
        and wavenum_max is None
    ):
        raise ValueError("Give wavenumber or wavelength")

    # check if only one pair of values has been passed
    w_present = wmin is not None or wmax is not None
    wavenum_present = wavenum_min is not None or wavenum_max is not None
    wavelength_present = wavelength_min is not None or wavelength_max is not None
    if w_present + wavenum_present + wavelength_present >= 2:
        raise ValueError("Cannot pass more than one set of values as input")
    assert medium in ["air", "vacuum"]

    # Get all waveranges

    if wmin is not None and wmax is not None:
        assert wavenum_min is None and wavenum_max is None
        assert wavelength_min is None and wavelength_max is None
        
        if represent_wavelength:
            wavelength_min = wmin
            wavelength_max = wmax
        elif represent_wavenum:
            wavenum_min = wmin
            wavenum_max = wmax

    # ... Input is in wavelength:
    if wavenum_min is None and wavenum_max is None:
        assert wavelength_max is not None
        assert wavelength_min is not None
        # Test range is correct:
        assert wavelength_min < wavelength_max

        # In wavelength mode, the propagating medium matters. Convert to
        # calculation medium (vacuum) if needed:
        if medium == "air":
            wavelength_min_vac = air2vacuum(wavelength_min)
            wavelength_max_vac = air2vacuum(wavelength_max)
        else:
            wavelength_min_vac = wavelength_min
            wavelength_max_vac = wavelength_max

        wavenum_min = nm2cm(wavelength_max_vac)
        wavenum_max = nm2cm(wavelength_min_vac)
    # ... or input is in wavenumber:
    else:
        assert wavenum_min is not None
        assert wavenum_max is not None
        # Test range is correct:
        assert wavenum_min < wavenum_max

    return wavenum_min, wavenum_max


In [107]:
get_waverange(1200 * u.cm, 1400 * u.cm)

(1680.0, 1440.0)

In [108]:
get_waverange(wavelength_min=1200 * u.cm, wavelength_max=1400 * u.cm)

(1680.0, 1440.0)

In [109]:
get_waverange(1200 * u.cm**-1, 1400 * u.cm**-1)

(1200.0, 1400.0)

In [110]:
get_waverange(wavenum_min = 1200, wavenum_max=1400)

(1200, 1400)

In [111]:
get_waverange(1200, 1400, wunit='nm')

(0.000168, 0.000144)

In [112]:
get_waverange(wavelength_min=1200, wavelength_max=1400) 

(0.000168, 0.000144)

In [113]:
get_waverange(1200, 1400, wunit='1/cm')

(1200.0, 1400.0)

In [114]:
get_waverange(wavenum_min=1200, wavenum_max=1400)

(1200, 1400)

In [116]:
get_waverange(1200*u.cm, 1000*u.cm, wunit = "1/m")

ValueError: Cannot pass units with both wavelength and wave unit

In [119]:
get_waverange(1200,1400)

AssertionError: 